<a href="https://colab.research.google.com/github/ryanraba/casa6/blob/master/CASA6_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
print("installing pre-requisite packages...")
!apt-get install python-casacore > /dev/null
!apt-get install libgfortran3 > /dev/null
!mkdir wheels > /dev/null
import urllib.request

print("downloading wheels...")
urllib.request.urlretrieve("https://www.cv.nrao.edu/~rraba/wheels/casa6wheels.tar.gz", "wheels/casa6wheels.tar.gz")

print("extracting and installing...")
!tar -xvf wheels/casa6wheels.tar.gz -C wheels/ > /dev/null
!rm wheels/casa6wheels.tar.gz
!pip install --no-index --find-links wheels/ -r wheels/requirements.txt > /dev/null

print("downloading MeasurementSet from CASAguide First Look at Imaging")
urllib.request.urlretrieve("https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated_flagged.ms.tar", "sis14_twhya_calibrated_flagged.ms.tar")

!tar -xvf sis14_twhya_calibrated_flagged.ms.tar > /dev/null
!rm sis14_twhya_calibrated_flagged.ms.tar

In [7]:
from casatasks import listobs

rc = listobs(vis='sis14_twhya_calibrated_flagged.ms')

!tail casa-*.log

2019-03-28 19:52:30	INFO	listobs::ms::summary+	  15   DV10  A071      12.0 m   -067.45.19.9  -22.53.23.5        -60.7887     -563.2541       23.3799  2225011.141945 -5440147.560932 -2481557.855663
2019-03-28 19:52:30	INFO	listobs::ms::summary+	  16   DV13  A072      12.0 m   -067.45.12.6  -22.53.24.0        147.1742     -580.5887       18.1825  2225199.254375 -5440058.161494 -2481571.803699
2019-03-28 19:52:30	INFO	listobs::ms::summary+	  17   DV15  A074      12.0 m   -067.45.12.1  -22.53.32.0        161.8159     -828.6196       18.7688  2225176.483514 -5439963.820451 -2481800.529842
2019-03-28 19:52:30	INFO	listobs::ms::summary+	  18   DV16  A069      12.0 m   -067.45.21.3  -22.53.30.2       -101.4797     -770.1047       23.2972  2224942.993176 -5440088.421459 -2481748.384855
2019-03-28 19:52:30	INFO	listobs::ms::summary+	  19   DV17  A138      12.0 m   -067.45.17.1  -22.53.34.4         19.1461     -901.2603       26.0137  2225036.269025 -5439997.853009 -2481870.267607
2019-03-28 19:5